# Classificador de assuntos de um texto
Classificador de assuntos que considera a distancia entre as palavras procuradas

Por considerar a distância e não a sequência exata das palavras no texto, como é feito no classificador
baseado em busca em profundidade, este classificador consegue lidar com palavras que não estão em ordem sequencial e até mesmo com a presença de outras palavras na frase que não interfiram no entendimento do assunto.

Devido a esta maleabilidade, pode ser necessário, em alguns assuntos, definir uma lista de palavras que estando na frase ou na vizinhança, negativem o resultado.


In [1]:
# importa do módulo texto a função que normaliza o conteúdo,
# retirando acentuação e palavras que não interferem no sentido da frase
from lib.texto import normaliza

### 1ª Etapa
Normalizar o texto que será classificado, retirando acentos e palavras descartáveis e converter o texto em uma lista de palavras

In [2]:
# texto que será classificado
texto = '''Tendo em vista a LIQUIDAÇÃO TOTAL
DO DÉBITO representado pela (s) CDA(s) acima
indicada(s), requer a Fazenda Pública do Estado de São Paulo,
por seu (sua) Procurador(a), abaixo assinado (a),
requer a extinção do  feito, com fundamento no artigo
924, inciso II, do Código de Processo Civil.
Caso deferida a pretensão supra, dá-
se, desde já, por ciente.
Pede deferimento   924.'''


In [3]:
texto = normaliza(texto)
palavras = texto.split(' ')
print('Texto padronizado e convertido em uma lista de palavras:')
print(palavras)

Texto padronizado e convertido em uma lista de palavras:
['tendo', 'vista', 'liquidacao', 'total', 'debito', 'representado', 'cdas', 'acima', 'indicadas', 'requer', 'fesp', 'procuradora', 'abaixo', 'assinado', 'requer', 'extincao', 'feito', 'fundamento', 'art', '924', 'dois', 'codigo', 'processo', 'civil', 'caso', 'deferida', 'pretensao', 'supra', 'ciente', 'pede', 'deferimento', '924']


### 2ª Etapa
Gerar as listas de assuntos a partir do dicionário de dados padronizado que contém os campos:
* `id`: Número único de identificação do assunto
* `titulo`: Descrição do assunto
* `contem`: lista com os padrões contidos no assunto. Cada item da lista deve ser:
    * uma string contendo palavras e/ou listas de palavras sinônimas `[(extincao|extinto)]`
    * esta string deve ser convertida em uma lista de palavras e/ou uma lista de palavras sinônimas
* `nao_contem`: lista com as palavras que não podem estar na frase encontrada nem na vizinhança

As palavras ou frases carregadas no dicionário padrão já devem ter sido padronizadas conforme as regras da função `normaliza()`

In [4]:
# dicionario contendo os tipos de assuntos para busca -> base para montagem das listas de busca
# Para simular registros em um Banco de dados, cada linha é um string
# As palavras sinônimas ficam dentro de um formato padrão [(extincao|extinto)]
assuntos_dic = {
    1: {
        'id': 1,
        'titulo': 'pedido extincao art. 924,ii',
        'contem': [
            'requer extincao [(execucao|processo|feito)] 924 dois',
            'requer [(extincao|extinto)] [(execucao|processo|feito)] 924 dois',
        ],
        'nao_contem': ['nao', 'revogacao']
    },
    2: {
        'id': 2,
        'titulo': 'revogacao sentenca',
        'contem': [
            'requer revogacao sentenca extincao [(execucao|processo|feito)]',
        ]
    },
    3: {
        'id': 3,
        'titulo': 'termo de ciencia',
        'contem': [
            'supra ciente',
        ]
    },
}

Para ser usado no classificador, o dicionário acima (`assuntos_dic`) tem que ser convertido na lista que segue:

In [5]:
# exemplo de como deve ficar a lista convertida a partir do dicionario de assuntos
# itens de cada linha: 0->id; 1->contem e/ou; 2->não contém
assuntos_lista = [
    [1, ['requer', 'extincao', ['execucao', 'processo', 'feito'], '924', 'dois'], ['nao', 'revogacao']],
    [1, ['requer', ['extincao', 'extinto'], ['execucao', 'processo', 'feito'], '924', 'dois'], ['nao', 'revogacao']],
    [2, ['requer', 'revogacao', 'sentenca', 'extincao', ['execucao', 'processo', 'feito']], []],
    [3, ['supra', 'ciente'],[]],
]           

In [6]:
# A função _prepara_lista_e_ou faz conversão da linha
# recebe uma string e a converte numa lista de palavras E/OU
# conforme o modelo acima
def _prepara_lista_e_ou(texto):
    marca1 = '[('
    marca2 = ')]'
    if not(marca1 in texto):
        return texto.split(' ')
    if texto.count(marca1) != texto.count(marca2):
        msg = 'Erro no formato de\n{}\n quantidade de "[(" e ")]" diferentes'.format(texto)
        raise ValueError(msg)
    lista = []
    while True:
        pos1 = texto.find(marca1)
        if pos1 < 0:
            if not texto:
                break
            for palavra in texto.split(' '):
                lista.append(palavra)
            break
        pos2 = texto.find(marca2)
        if pos1 != 0:
            tmp = texto[:pos1].strip()
            for palavra in tmp.split(' '):
                lista.append(palavra)
        pos2 += 2
        texto_ou = texto[pos1 + 2:pos2 - 2]
        lista.append(texto_ou.split('|'))
        texto = texto[pos2:].strip()
    return lista

In [7]:
_prepara_lista_e_ou('supra ciente')

['supra', 'ciente']

In [8]:
_prepara_lista_e_ou('requer extincao [(execucao|processo|feito)] 924 dois')

['requer', 'extincao', ['execucao', 'processo', 'feito'], '924', 'dois']

In [9]:
_prepara_lista_e_ou('requer revogacao sentenca extincao [(execucao|processo|feito)]')

['requer',
 'revogacao',
 'sentenca',
 'extincao',
 ['execucao', 'processo', 'feito']]

In [10]:
_prepara_lista_e_ou('requer [(extincao|extinto)] [(execucao|processo|feito)] 924 dois')

['requer',
 ['extincao', 'extinto'],
 ['execucao', 'processo', 'feito'],
 '924',
 'dois']

In [11]:
# A função _prepara_lista_de_busca recebe um item do dicionário padronizado
# e converte este item em listas de assuntos.
# cada linha da lista é uma lista com os campos: 0->id; 1->contem e/ou; 2->não contém
def _prepara_lista_de_busca(dic_item):
    listas = []
    if not('contem' in dic_item):
        raise KeyError('Chave contem nao encontrada no dicionario')
    if not('id' in dic_item):
        raise KeyError('Chave id nao encontrada no dicionario')
    for linha in dic_item['contem']:
        lista=[]
        lista.append(dic_item['id'])
        lista.append(_prepara_lista_e_ou(linha))
        if 'nao_contem' in dic_item:
            lista.append(dic_item['nao_contem'])
        else:
            lista.append([])
        listas.append(lista)
    return listas

In [12]:
# testes de _prepara_lista_de_busca()
print(_prepara_lista_de_busca(assuntos_dic[1]))

[[1, ['requer', 'extincao', ['execucao', 'processo', 'feito'], '924', 'dois'], ['nao', 'revogacao']], [1, ['requer', ['extincao', 'extinto'], ['execucao', 'processo', 'feito'], '924', 'dois'], ['nao', 'revogacao']]]


In [13]:
print(_prepara_lista_de_busca(assuntos_dic[3]))

[[3, ['supra', 'ciente'], []]]


In [14]:
# A partir do dicionário padrão, as listas são preparadas e extraídas
# para serem utilizadas no classificador de assuntos
def converte_dicionario_em_listas_de_busca(dic):
    listas = []
    for i in dic.keys():
        lista = _prepara_lista_de_busca(dic[i])
        for item in lista:
            listas.append(item)
    return listas

In [15]:
print(converte_dicionario_em_listas_de_busca(assuntos_dic))

[[1, ['requer', 'extincao', ['execucao', 'processo', 'feito'], '924', 'dois'], ['nao', 'revogacao']], [1, ['requer', ['extincao', 'extinto'], ['execucao', 'processo', 'feito'], '924', 'dois'], ['nao', 'revogacao']], [2, ['requer', 'revogacao', 'sentenca', 'extincao', ['execucao', 'processo', 'feito']], []], [3, ['supra', 'ciente'], []]]


In [16]:
# faz a comparação entre a lista gerada manualmente, o esperado, com a lista gerada pela função
# _converte_dicionario_em_listas_de_busca()
assuntos_lista == converte_dicionario_em_listas_de_busca(assuntos_dic)

True

### 3ª Etapa
O núcleo do classificador são as funções que trabalham com cada uma das linhas geradas pela função `_converte_dicionario_em_listas_de_busca(): [1, ['requer', 'extincao', ['execucao', 'processo', 'feito'], '924', 'dois'], ['nao', 'revogacao']]` procurando identificar a sequencia de palavras vizinhas que mais se aproximam com as listas padrão (segunda posição da lista) e que não possuem na vizinhança nenhuma das palavras que negativam o assunto (terceira posição da lista)

A função `_procura_lista_de_palavras(lista, palavras)` usa internamente a função `_consulta_palavra(palavra, palavras)` e retorna uma listagem com as posicões encontradas de cada uma das palavras distintas, ou pelo menos uma das palavras sinônimas contidas no texto.

A lista retornada pela função `_procura_lista_de_palavras()` é transformada numa matriz com as posicoes entre as palavras encontradas e esta é usada para calcular a distância média entre as palavras e encontrar a sequência de palavras, se houver, que identificam a presença do assunto pesquisado.

In [17]:
# faz a consulta de uma palavra no texto (convertido em uma lista)
# e retorna uma lista com as posições encontradas da palavra
# ou -1 se a palavra não estiver no texto
def _consulta_palavra(palavra, palavras):
    if not palavra in palavras:
        return -1
    lista = []
    inicio = 0
    for i in range(palavras.count(palavra)):
        pos = palavras.index(palavra, inicio)
        lista.append(pos)
        inicio = pos + 1
    return lista

In [18]:
# procura uma lista de palavras no texto e retorna todas
# as posições encontradas, se uma das palavras da lista não for encontrada
# retorna uma lista vazia
def _procura_lista_de_palavras(lista, palavras):
    result = []
    for item in lista[1]:
        if isinstance(item, str):
            # o item é uma palavra, ela tem que estar presente no texto
            posicoes = _consulta_palavra(item, palavras)
        else:
            # o item é uma lista de palavras sinonimas, pelo menos uma tem que ser encontrada
            posicoes = []
            for palavra in item:
                posicao = _consulta_palavra(palavra, palavras)
                if posicao == -1:
                    continue
                posicoes += posicao
            if not posicoes:
                posicoes = -1
        if posicoes == -1:
            return []
        result.append(posicoes)
    return [lista[0], result]

In [19]:
# teste da consulta de cada uma das palavras de uma lista:
lista = assuntos_lista[0][1][2]
print(lista)
for palavra in lista:
    print(palavra, _consulta_palavra(palavra, palavras))

['execucao', 'processo', 'feito']
execucao -1
processo [22]
feito [16]


In [20]:
# lista onde todas as palavras estao no texto
print(palavras)
print(assuntos_lista[0])
esperado = [1, [[9, 14], [15], [22, 16], [19, 31], [20]]]
resultado = _procura_lista_de_palavras(assuntos_lista[0], palavras)
print(resultado)
esperado == resultado

['tendo', 'vista', 'liquidacao', 'total', 'debito', 'representado', 'cdas', 'acima', 'indicadas', 'requer', 'fesp', 'procuradora', 'abaixo', 'assinado', 'requer', 'extincao', 'feito', 'fundamento', 'art', '924', 'dois', 'codigo', 'processo', 'civil', 'caso', 'deferida', 'pretensao', 'supra', 'ciente', 'pede', 'deferimento', '924']
[1, ['requer', 'extincao', ['execucao', 'processo', 'feito'], '924', 'dois'], ['nao', 'revogacao']]
[1, [[9, 14], [15], [22, 16], [19, 31], [20]]]


True

In [21]:
print(assuntos_lista[3])
resultado = _procura_lista_de_palavras(assuntos_lista[3], palavras)
print(resultado)

[3, ['supra', 'ciente'], []]
[3, [[27], [28]]]


In [22]:
print(palavras)
lista1 = [1, ['requer', 'extincao', ['execucao', 'processo', 'feito'], '924', 'dois'], ['nao', 'revogacao']]
esperado = [1, [[9, 14], [15], [22, 16], [19, 31], [20]]]
resultado = _procura_lista_de_palavras(lista1, palavras)
print(resultado)
esperado == resultado

['tendo', 'vista', 'liquidacao', 'total', 'debito', 'representado', 'cdas', 'acima', 'indicadas', 'requer', 'fesp', 'procuradora', 'abaixo', 'assinado', 'requer', 'extincao', 'feito', 'fundamento', 'art', '924', 'dois', 'codigo', 'processo', 'civil', 'caso', 'deferida', 'pretensao', 'supra', 'ciente', 'pede', 'deferimento', '924']
[1, [[9, 14], [15], [22, 16], [19, 31], [20]]]


True

In [23]:
# lista contendo pelo menos uma palavra que não está no texto
print(assuntos_lista[2])
esperado = []
resultado = _procura_lista_de_palavras(assuntos_lista[2], palavras)
print(resultado)
esperado == resultado

[2, ['requer', 'revogacao', 'sentenca', 'extincao', ['execucao', 'processo', 'feito']], []]
[]


True

In [24]:
# transforma a lista das posicoes das palavras encontradas
# em uma matriz com as sequencias possiveis das palavras
# A partir desta matriz a distância média entre as palavras é calculada
def _prepara_matriz_dos_resultados(posicoes):
    dic = {}
    for i in range(len(posicoes[0])):
        primeiro = posicoes[0][i]
        dic[i] = [[primeiro]]
        for listas in posicoes[1:]:
            for j in range(len(listas)):
                item = listas[j]
                if j == 0:
                    for l in dic[i]:
                        l.append(item)
                else:
                    for l in range(len(dic[i])):
                        tmp = dic[i][l]
                        tmp = tmp[:-1]
                        tmp.append(item)
                        dic[i].append(tmp)
    resultado = []
    for lista in dic.values():
        resultado += lista
    return resultado

In [25]:
# calcula a distância média entre as palavras
# retornando a linha com menor distância e o valor da distância média
def _retorna_palavras_mais_proximas(matriz):
    mais_proximo = [[], 100000]
    for lista in matriz:
        lista.sort()
        dist = 0
        anterior = lista[0]
        for i in range(1, len(lista)):
            atual = lista[i]
            dist += atual - anterior
            anterior = atual
        dist_media = 1
        if len(lista) > 1:
            dist_media = float(dist) / (len(lista) -1)
            dist_media = float(("%.2f" % dist_media))
        # print(lista, dist, dist_media)
        if dist_media < mais_proximo[1]:
            mais_proximo[0] = lista
            mais_proximo[1] = dist_media
    return mais_proximo

In [26]:
result = [[9, 14], [15], [16], [19, 31], [20]]
'''
lista: [[9, 14], [15], [16], [19, 31], [20]]
matriz:
[[9, 15, 16, 19, 20], [9, 15, 16, 31, 20], [14, 15, 16, 19, 20], [14, 15, 16, 31, 20]]
mais proximas, distancia média:
[14, 15, 16, 19, 20], 1.5
'''
matriz = _prepara_matriz_dos_resultados(result)
print(matriz)
_retorna_palavras_mais_proximas(matriz)

[[9, 15, 16, 19, 20], [9, 15, 16, 31, 20], [14, 15, 16, 19, 20], [14, 15, 16, 31, 20]]


[[14, 15, 16, 19, 20], 1.5]

In [27]:
result = [[10, 20], [11, 21], [13, 22], [24]]
'''
matriz:
[[10, 11, 13, 24], [10, 21, 13, 24], [10, 11, 22, 24], [10, 21, 22, 24], [20, 11, 13, 24], [20, 21, 13, 24], [20, 11, 22, 
24], [20, 21, 22, 24]]
mais proximas, distancia média:
[20, 21, 22, 24], 1.33
'''
matriz = _prepara_matriz_dos_resultados(result)
print(matriz)
_retorna_palavras_mais_proximas(matriz)

[[10, 11, 13, 24], [10, 21, 13, 24], [10, 11, 22, 24], [10, 21, 22, 24], [20, 11, 13, 24], [20, 21, 13, 24], [20, 11, 22, 24], [20, 21, 22, 24]]


[[20, 21, 22, 24], 1.33]

In [28]:
result = [[9, 14], [15], [16], [20, 31], [32]]
'''
matriz:
[[9, 15, 16, 20, 32], [9, 15, 16, 31, 32], [14, 15, 16, 20, 32], [14, 15, 16, 31, 32]]
mais proximas, distancia média:
[14, 15, 16, 20, 32], 4.5
'''
matriz = _prepara_matriz_dos_resultados(result)
print(matriz)
_retorna_palavras_mais_proximas(matriz)

[[9, 15, 16, 20, 32], [9, 15, 16, 31, 32], [14, 15, 16, 20, 32], [14, 15, 16, 31, 32]]


[[14, 15, 16, 20, 32], 4.5]

In [29]:
result = [[39, 58], [41], [43], [10, 44], [50]]
'''
matriz:
[[39, 41, 43, 10, 50], [39, 41, 43, 44, 50], [58, 41, 43, 10, 50], [58, 41, 43, 44, 50]]
mais proximas, distancia média:
[39, 41, 43, 44, 50], 2.75
'''
matriz = _prepara_matriz_dos_resultados(result)
print(matriz)
_retorna_palavras_mais_proximas(matriz)

[[39, 41, 43, 10, 50], [39, 41, 43, 44, 50], [58, 41, 43, 10, 50], [58, 41, 43, 44, 50]]


[[39, 41, 43, 44, 50], 2.75]

In [30]:
result = [[40, 58], [41, 59], [43], [10, 44], [45, 60]]
'''
matriz:
[[40, 41, 43, 10, 45], [40, 59, 43, 10, 45], [40, 41, 43, 44, 45], [40, 59, 43, 44, 45], [40, 41, 43, 10, 60],[40, 59, 
43, 10, 60], [40, 41, 43, 44, 60], [40, 59, 43, 44, 60], [58, 41, 43, 10, 45], [58, 59, 43, 10, 45], [58, 41, 43, 44, 45],
[58, 59, 43, 44, 45], [58, 41, 43, 10, 60], [58, 59, 43, 10, 60], [58, 41, 43, 44, 60],[58, 59, 43, 44, 60]]
mais proximas, distancia média:
[40, 41, 43, 44, 45], 1.25
'''
matriz = _prepara_matriz_dos_resultados(result)
print(matriz)
_retorna_palavras_mais_proximas(matriz)

[[40, 41, 43, 10, 45], [40, 59, 43, 10, 45], [40, 41, 43, 44, 45], [40, 59, 43, 44, 45], [40, 41, 43, 10, 60], [40, 59, 43, 10, 60], [40, 41, 43, 44, 60], [40, 59, 43, 44, 60], [58, 41, 43, 10, 45], [58, 59, 43, 10, 45], [58, 41, 43, 44, 45], [58, 59, 43, 44, 45], [58, 41, 43, 10, 60], [58, 59, 43, 10, 60], [58, 41, 43, 44, 60], [58, 59, 43, 44, 60]]


[[40, 41, 43, 44, 45], 1.25]

### 4ª Etapa

O classificador recebe o texto e o **dicionário padronizado** ou a **lista de assuntos** já formatada onde o primeiro índice de cada item é o código, segundo é a lista de palavras contidas no assunto e o terceiro a lista de palavras que não deve estar presente no texto encontrado ou vizinhança.

A distância média mínima necessária entre as palavras encontradas tem como valor padrão 2.

Basicamente o que fica "exposto" no classificador são as funções `converte_dicionario_em_listas_de_busca(assuntos)` e `classifica(texto, assuntos, distancia)`.

In [31]:
def classifica(texto, assuntos, distancia=2):
    if isinstance(assuntos, dict):
        assuntos = converte_dicionario_em_listas_de_busca(assuntos)
    texto = normaliza(texto)
    palavras = texto.split(' ')
    retorno = {}
    for lista in assuntos:
        result = _procura_lista_de_palavras(lista, palavras)
        if not result:
            continue
        matriz = _prepara_matriz_dos_resultados(result[1])
        mais_proximos = _retorna_palavras_mais_proximas(matriz)
        if mais_proximos[1] <= distancia:
            if result[0] in retorno.keys():
                if mais_proximos[1] <= retorno[result[0]][1]:
                    del(retorno[result[0]])
                # print('ja existe')
            inicio = mais_proximos[0][0]
            fim = mais_proximos[0][-1] + 1
            texto = ' '.join(palavras[inicio:fim])
            tem_restricao = False
            n_vizinhos = distancia + 1
            if lista[2]:
                if inicio <=  n_vizinhos:
                    inicio = 0
                else:
                    inicio -=  n_vizinhos
                fim +=  n_vizinhos
                texto_vizinho = ' '.join(palavras[inicio:fim])
                exclusoes = lista[2]
                for excluir in exclusoes:
                    if excluir in texto_vizinho:
                        tem_restricao = True
                        break
            if not tem_restricao:
                retorno[result[0]] = [texto, mais_proximos[1]]
    return retorno


In [32]:
# Neste texto, não há pedido de extinção nos termos do artigo 924, II
# e sim um pedido de revogação da sentença
texto5 = 'tendo a informacao da fesp sobre o cancelamento da dívida, requer a revogação da sentença \
de extinção do processo com fundamento  no art. 924, II do CPC e requer a extinção nos termos do art. 26 da LEF'
normaliza(texto5)

'tendo informacao fesp cancelamento divida requer revogacao sentenca extincao processo fundamento art 924 dois cpc requer extincao termos art 26 lef'

In [33]:
# isoladamente, os dois pedidos são validados
palavras5 = normaliza(texto5).split(' ')
lista1 = [1, ['requer', 'extincao', ['execucao', 'processo', 'feito'], '924', 'dois'], ['nao', 'revogacao']]
# ('resultado', [1, [[5, 15], [8, 16], [[9]], [12], [13]]])
result = _procura_lista_de_palavras(lista1, palavras5)
print(result)

[1, [[5, 15], [8, 16], [9], [12], [13]]]


In [34]:
lista2 = [2, ['requer', 'revogacao', 'sentenca', 'extincao', ['execucao', 'processo', 'feito']]]
# ('resultado', [2, [[5, 15], [6], [7], [8, 16], [9]]])
result2 = _procura_lista_de_palavras(lista2, palavras5)
print(result2)

[2, [[5, 15], [6], [7], [8, 16], [9]]]


In [35]:
# o classificar deve saber desconsiderar o primeiro resultado e retornar apenas o segundo
classifica(texto5, assuntos_lista)

{2: ['requer revogacao sentenca extincao processo', 1.0]}

In [36]:
# Neste texto, não há pedido de extinção nos termos do artigo 924, II
texto6 = 'dívida, requer a extinção do processo com fundamento  no art. 924, III do CPC, requer'
normaliza(texto6)

'divida requer extincao processo fundamento art 924 tres cpc requer'

In [37]:
classifica(texto6, assuntos_lista)

{}

In [38]:
# no texto original, deve ser encontrado o pedido de extinção por pagamento e o termo de ciência
print(texto)
classifica(texto, assuntos_lista)

tendo vista liquidacao total debito representado cdas acima indicadas requer fesp procuradora abaixo assinado requer extincao feito fundamento art 924 dois codigo processo civil caso deferida pretensao supra ciente pede deferimento 924


{1: ['requer extincao feito fundamento art 924 dois', 1.5],
 3: ['supra ciente', 1.0]}

In [39]:
# um texto parecido com o original mas com as palavras em ordem diferente
texto1 = '... a procuradora abaixo assinado requer, com fundamento no art. 924, II do CPC, a extinção do processo \
caso deferida pretensao supra ciente. pede deferimento .'

In [40]:
classifica(texto1, assuntos_lista)

{1: ['requer fundamento art 924 dois cpc extincao processo', 1.75],
 3: ['supra ciente', 1.0]}

In [41]:
# basicamente o mesmo texto, mas recebendo o dicionário e não a lista já preparada
texto2 = 'tendo vista liquidacao total debito representado cdas acima indicadas fazenda publica estado sao \
paulo procuradora abaixo assinado com fundamento no art. 924, II do CPC, requer processo extinto \
caso deferida pretensao supra ciente. pede deferimento .'
classifica(texto2, assuntos_dic)

{1: ['924 dois cpc requer processo extinto', 1.25], 3: ['supra ciente', 1.0]}

In [42]:
texto3 = '''Tendo em vista a LIQUIDAÇÃO TOTAL
DO DÉBITO representado pela (s) CDA(s) acima
indicada(s), a Fazenda Pública do Estado de São Paulo,
por seu (sua) Procurador(a), abaixo assinado (a),
requer a extinção do  feito, com fundamento e na forma do artigo
924, inciso II, do Código de Processo Civil.   .'''

In [43]:
classifica(texto3, assuntos_lista)

{1: ['requer extincao feito fundamento forma art 924 dois', 1.75]}